# Imports

In [1]:
import os

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_size = 1_800 # 6_660
inicial_date = "11/5/2022"    # When database beging
rows_per_person =  (3 * 6)    # Num of people * Num of meals a day

archive = "Final Proyect 2.xlsx"         # Main archive
second_archive = "Data _ new.xlsx"     # Second archive

### Key df parameters 

In [4]:
parameters = [
    'calories', 
    'grasas saturada (sum)', 
    'grasas insaturadas (sum)', 
    'proteinas (sum)', 
    'carbohidratos'
    ]

### Excel sheets

In [5]:
all_sheets = [
 "Data",
 "Calories",                 
 "fat saturated",        
 "fat insaturated",      
 "proteins",  
 'carbohidratos' 
]

# Calorias de los alimentos

In [6]:
def meals():
    return {
        "cereal": 120,               # Per cup
        "tortilla de maiz": 50,      # Per tortilla
        "tortilla de nopal": 30,     # Per tortilla
        "palomitas de maiz": 35,     # Per cup
        "papa cocida": 90,           # Per medium potato
        "camote": 100,               # Per medium sweet potato
        "pan de caja orowheat": 70,  # Per slice
        "pan tostado": 65,           # Per slice
        "galleta de avena": 50,      # Per cookie
        "tostada horneada": 70,      # Per piece
        "galletas marias": 30,       # Per cookie
        "galletas habaneras": 40,    # Per cookie
        "avena": 150,                # Per cup cooked
        "grano lamixta": 120,        # Per cup
        "salmitas": 160,             # Per packet
        "arroz": 200,                # Per cup cooked
        "pasta": 220,                # Per cup cooked
        "yogurt griego": 100,        # Per 100g
        "frijoles molidos": 120,     # Per cup
        "habas": 110,                # Per cup
        "pechuga de pollo": 165,     # Per 100g
        "clara de huevo": 17,        # Per egg white
        "sobre de atun": 90,         # Per 100g
        "pechuga de pavo": 120,      # Per 100g
        "salmon ahumado": 120,       # Per 100g
        "pescado": 200,              # Per 100g
        "queso panela light": 90,    # Per 100g
        "bistec de bola cortemagro": 150,    # Per 100g
        "huevo": 78,                         # Per large egg
        "carne molida de sirloin": 200,      # Per 100g
        "filete de res": 250,        # Per 100g
        "filete de arrachera": 210,  # Per 100g
        "aguacate(con cucharadita aceite de coco)": 120,  # Per 100g
        "nueces": 650,               # Per 100g
        "almendras": 579,            # Per 100g
        "aceite de aguacate": 884,   # Per 100g
        "aceite de oliva": 884,      # Per 100g
        "mantequilla de maní": 590,  # Per 100g
        "Melon": 34,                 # Per 100g
        "Platano": 89,               # Per 100g
        "Manzana": 52,               # Per 100g
        "Chapata": 250,              # Per loaf
        "Salmon": 206,               # Per 100g
        "Pasta": 220,                # Already defined
        "Arroz": 200,                # Already defined
        "Frituras": 536,             # Per 100g
        "Tacos": 150,                # Per taco
        "Motuleños": 400,            # Per plate
        "Tostada": 70,               # Already defined
        "Uvas": 69,                  # Per 100g
        "Pure": 85,                  # Per 100g
        "Tacos de salmon": 200,      # Per taco
        "Café": 2,                   # Per cup
        "Chocolate": 546,            # Per 100g
        "Snacks": 500,               # Per 100g
        "Pizza": 266,                # Per slice
        "sabritas": 536,             # Per 100g
        "fuzetea": 19}               # Per 100ml

# Generate dates

In [7]:
def generate_dates(inicial_date: str = inicial_date, size: int = data_size, rows: int = rows_per_person) -> list:
    month, day, year = [int(date) for date in inicial_date.split("/")]

    dates = [f"{month}/{day}/{year}" for _ in range(rows)]
    
    for _ in range(size // rows - 1):
        month31 = month == 1 or month == 3 or month == 5 or month == 7 or month == 8 or month == 10
        month30 = month == 4 or month == 6 or month == 9 or month == 10 or month == 11

        if month == 12 and day == 31:
            
            year += 1
            day, month = 1, 1

        elif month31 and day == 31 or month30 and day == 30 or month == 2 and day == 28:  # not prepare for leap years

            month += 1
            day = 1

        else:
            
            day += 1


        dates += [f"{month}/{day}/{year}" for _ in range(rows)]
        

    return dates

# Generate parameters

In [26]:
def index_match(
        data: str, row: int, page_2: str, 
        page_1: str = all_sheets[0], buttom: str = "I", amount_of_formulas: int = len(meals()),abcd = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
    

    def _xlabel(buttom: str, top: int):
    
        buttom = {
            key: value + 1 
            for value, key in enumerate(abcd)
            }[buttom]
        
        top += buttom     # check later -----------

        basic_xlabel = abcd
        xlabel = [letter for letter in abcd]

        for letter in basic_xlabel:
            for secondletter in basic_xlabel:
                xlabel.append(letter + secondletter)

        return xlabel[buttom:top]


    xlabel = _xlabel(buttom, amount_of_formulas)
    formulas = "= "
    
    page_1 = "Data"
    coordenates1 = "$1"           # row lookup 
    coordenates2 = "2"            # col Lookup
    
    page_2 = "Calories"
    coordenates = "$A:$L"         # Main array---------------

    coordenates3 = "$A$1:$L$1"    # col array 
    coordenates4 = "$A:$A"        # row array 
    
    
    
    for i, cell in enumerate(xlabel):
        if i == (amount_of_formulas - 1):
            formulas += f"INDEX({page_2}!$A$1:$BF$11, {page_1}!${cell}{row} + 2,  {i + 1})"
        else:
            formulas += f"INDEX({page_2}!$A$1:$BF$11, {page_1}!${cell}{row} + 2,  {i + 1}) + "
            

    # INDEX(Calories!$A$1:$BF$11, Data!$J2 + 2,  1)

    # INDEX(Calories!$A$1:$BF$11, Data!$J3 + 2,  1)
            
            
    # INDEX(Calories!A:L, MATCH(Data!B$1, Calories!$A:$A, 0),MATCH(Data!B2, Calories!$A$1:$L$1,0 ))
            
    # formulas = f"Index({page_2}!{coordenates}, Match({page_1}!{coordenates1}, {page_2}!{coordenates3}, 0), Match({page_1}!{coordenates2}, {page_2}!{coordenates4}, 0))"
    
    return formulas

# Generate data

In [9]:
def generate_data(data: str) -> pd.DataFrame:

    if data == all_sheets[0]:
        
        total_hours = []
        people = ['palomares' , 'adrian', 'angel'] * (data_size // 3)
        keys = [
                    'fecha', 'hora', 'persona', 'num. comidas'
                ]
        hours = [
                    '7:00:00 AM',
                    '10:00:00 AM',
                    '2:00:00 PM',
                    '6:00:00 PM',
                    '10:00:00 PM',
                    '11:00:00 PM'
    ]
        food = list(meals())

        
    # For filling in the calorie content for the newly added foods, I will use standard calorie values from reliable sources.
    # These are approximate values and may vary based on specific food varieties and preparation methods.
        amount_food = [[np.random.randint(0, 3) for _ in range(data_size)] for _ in enumerate(food)]


        for hour in hours: total_hours.extend((hour, hour, hour))
        total_hours *= (data_size // rows_per_person)

        values = [generate_dates(), total_hours, people] + [[np.nan for _ in range(data_size)] for _ in range(6)]
        

        df = {key: value for key, value in zip(keys, values)}
        for key, value in zip(food, amount_food): df[key] = value


        df =  pd.DataFrame(df)

        df['num. comidas'] = df[food].sum(axis=1, numeric_only=True)


        return df
    
    
    if data == all_sheets[1]:
        return pd.DataFrame({key: [value * i for i in range(10)] for key, value in meals().items()})
    
    if data == all_sheets[2]:
        return pd.DataFrame({key: [value * i for i in range(10)] for key, value in meals().items()})
    
    if data == all_sheets[3]:
        return pd.DataFrame({key: [value * i for i in range(10)] for key, value in meals().items()})
    
    if data == all_sheets[4]:
        return pd.DataFrame({key: [value * i for i in range(10)] for key, value in meals().items()})
    
    if data == all_sheets[5]:
        return pd.DataFrame({key: [value * i for i in range(10)] for key, value in meals().items()})

# Print data

In [27]:
def print_data(new: bool): # Add function to create a diferent archive ---- 

    with pd.ExcelWriter(archive ,engine="openpyxl") as writer:

        for sheet in all_sheets:
            generate_data(sheet).to_excel(writer, sheet_name=sheet, index=False)


        for i, key in enumerate(parameters):
            pd.DataFrame({key: [index_match(key, row + 2, page_2=all_sheets[i]) for row in range(data_size)]})\
                .to_excel(writer, sheet_name=all_sheets[0], startcol=(i + 4),index=False)

In [30]:
def read_new_data(new: bool):  # Add function to read a diferent archive ---- 
    
    return pd.read_excel(
        archive,
        sheet_name = all_sheets[0],
        na_values = None
    )

In [28]:
print_data(True)

In [33]:
os.remove(archive)

In [34]:
df = read_new_data(False)

df

FileNotFoundError: [Errno 2] No such file or directory: 'Final Proyect 2.xlsx'